In [5]:
import csv
from transformers import pipeline

ModuleNotFoundError: No module named 'transformers.utils'

In [2]:
data = "../Files/imdb_master.csv"

In [3]:
labels = {"neg": 0, "pos": 1}
texts = []

with open(data, encoding="ISO-8859-1") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        if not (row['label'] == "unsup"):
            texts.append((row['review'], labels[row['label']]))

texts = texts[int(len(texts) / 2) :]

In [4]:
len(texts)

25000